In [1]:
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
%matplotlib inline
import string
from wordcloud import WordCloud

import nltk
nltk.download('treebank')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.stem import PorterStemmer
#from nltk.stem.porter import PorterStemmer 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,classification_report
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('wordnet', quiet=True)
from contextlib import redirect_stdout
import os

import nltk
from nltk.corpus import wordnet


with redirect_stdout(open(os.devnull, "w")):
    nltk.download('wordnet')

[nltk_data] Downloading package treebank to C:\Users\Sai
[nltk_data]     Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sai
[nltk_data]     Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Sai
[nltk_data]     Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sai
[nltk_data]     Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sai Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Sai
[nltk_data]     Dhanush\AppData\Roaming\nltk_data...
[nltk_data]   Package 

In [2]:
data = pd.read_csv("sa_data.csv")
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [3]:
# Preprocessing - removing unwanted characters, tokenization, stop-word removal
def preprocess_data(tweet):
    user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
    has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
    url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
    
    x = tweet.lower()  # Lower casing all the characters
    x = re.sub(user_name, '', x) # Replace the username with an empty characrer
    x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
    x = re.sub(url, '', x) # Replace the URLs with an empty characrer
    x = re.sub("[^@a-zA-Z0-9]", " ",x) # Replace all the characters except alphabets and digits from the tweet
    x = re.sub("rt\s+",'',x) # Replace the retweet character "rt" from the tweet
    
    tokens = word_tokenize(x) # Tokenization; splitting the text into tokens
    port_stem = PorterStemmer()
    x=[] # to store stem words
    for word in tokens:
        x.append(port_stem.stem(word))
        

                          
    stop_words = list(set(stopwords.words("english")))
    negation_words = ["don't", "needn't", "won't", "wasn't", "didn't", "hadn't", "weren't", "not", "no", "hasn't", "nor", "shan't", "shouldn't", "isn't", "mustn't", "couldn't", "aren't", "haven't", "wouldn't", "doesn't", "mightn't"]
    stops = [w for w in stop_words if not w in negation_words]           
    words = [w for w in tokens if not w in stops] 

    return(" ".join(words)) # Combine the tokenized words to form a sentence

In [4]:
data['text'] = data['text'].apply(lambda x: preprocess_data(x))
data.head()

,text,sentiment
0,everyone feel climate change question last nig...,Neutral
1,catch full last night scott best lines 90 seconds,Positive
2,no mention tamir rice held cleveland wow,Neutral
3,carly fiorina trending hours debate men comple...,Positive
4,w delivered highest ratings history presidenti...,Positive


In [5]:
# Converting the dataframe into list
reviews = data['text'].tolist()
sentiment = data['sentiment'].tolist()

le = LabelEncoder()
#labels = le.fit_transform(sentiment)
Y = le.fit_transform(sentiment)

### Machine Learning with TF-IDF (Term Frequency - Inverse Document Frequency) features

In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) # TF-IDF feattres
#vectorizer = TfidfVectorizer() # TF-IDF feattres
X = vectorizer.fit_transform(reviews)
X = X.toarray()

In [7]:
print(vectorizer.get_feature_names())

['00', '00 11', '00 candidates', '00 crowd', '00 edt', '00 several', '000', '000 000', '000 1996', '000 across', '000 americans', '000 debt', '000 followers', '000 iowa', '000 iraqi', '000 jobs', '000 light', '000 new', '000 people', '000 protesters', '000 says', '000 sig', '000 subpoenaed', '000 votes', '000 women', '000 workers', '000 year', '00pm', '06', '06 bush', '07', '07 fun', '0hara', '10', '10 000', '10 11', '10 15', '10 2001', '10 amp', '10 angry', '10 appt', '10 best', '10 billion', '10 candidates', '10 contenders', '10 cringeworthy', '10 donation', '10 dreadful', '10 fit', '10 gotten', '10 gov', '10 great', '10 guys', '10 horsemen', '10 hrs', '10 htt', '10 idiots', '10 insane', '10 man', '10 many', '10 men', '10 mil', '10 million', '10 millionaires', '10 mins', '10 moments', '10 mudslinging', '10 people', '10 percent', '10 planned', '10 rand', '10 republican', '10 sec', '10 secs', '10 straight', '10 takes', '10 times', '10 trump', '10 tweet', '10 wealthy', '10 white', '10 w

In [8]:
# Splitting the data into train data and test data
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state = 1)
# Printing the size of the train data, train label, test data and test label
print("Shape train data = ",np.shape(X_train))
print("Shape of train label = ",np.shape(y_train))
print("Shape of test data = ",np.shape(X_test))
print("Shape of test label = ",np.shape(y_test))

Shape train data =  (11096, 58376)
Shape of train label =  (11096,)
Shape of test data =  (2775, 58376)
Shape of test label =  (2775,)


# Naive Bayes Classification

In [10]:
clf = GaussianNB() # Initializing the classifier
clf.fit(X_train, y_train) # Train the classifier using X_train and y_train

GaussianNB()

In [11]:
# Predict the class label for the test data using the trained model
y_pred = clf.predict(X_test)

In [12]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\n")
#fetching classification report
print(classification_report(y_test, y_pred))


[[899 471 349]
 [194 224 202]
 [121  57 258]]


              precision    recall  f1-score   support

           0       0.74      0.52      0.61      1719
           1       0.30      0.36      0.33       620
           2       0.32      0.59      0.41       436

    accuracy                           0.50      2775
   macro avg       0.45      0.49      0.45      2775
weighted avg       0.58      0.50      0.52      2775



# Implementing Logistic Regression

In [14]:
# import logistic regression model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [15]:
import warnings
warnings.filterwarnings('ignore')
logreg.fit(X_train,y_train)
# Predict the class label for the test data using the trained model
y_pred = logreg.predict(X_test)

In [16]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("\n")
#fetching classification report
print(classification_report(y_test, y_pred))


[[1628   53   38]
 [ 470  122   28]
 [ 268   28  140]]


              precision    recall  f1-score   support

           0       0.69      0.95      0.80      1719
           1       0.60      0.20      0.30       620
           2       0.68      0.32      0.44       436

    accuracy                           0.68      2775
   macro avg       0.66      0.49      0.51      2775
weighted avg       0.67      0.68      0.63      2775



# Implementing KNN, Hyper parameter tunning using GridSearch

In [18]:
knn=KNeighborsClassifier(n_neighbors=5)

In [19]:
#hyperparameter tuning
from sklearn.model_selection import GridSearchCV

#to  find suitable value of k
grid_param = {
    'n_neighbors': [3,5,7,9]
}

gd_sr = GridSearchCV(estimator=knn,
                     param_grid=grid_param,
                     scoring='accuracy')

In [20]:
gd_sr.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, scoring='accuracy')

In [21]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'n_neighbors': 5}


In [22]:
best_result = gd_sr.best_score_
print(best_result*100,"%")

42.510821772651425 %


In [23]:
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [24]:
y_pred=knn.predict(X_test)

In [25]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix is\n")
print(cm)

print("\n")
print(classification_report(y_test, y_pred))

print("Accuracy of model is",accuracy_score(y_test,y_pred)*100,"%")

confusion matrix is

[[ 626 1075   18]
 [ 118  489   13]
 [  65  279   92]]


              precision    recall  f1-score   support

           0       0.77      0.36      0.50      1719
           1       0.27      0.79      0.40       620
           2       0.75      0.21      0.33       436

    accuracy                           0.43      2775
   macro avg       0.60      0.45      0.41      2775
weighted avg       0.66      0.43      0.45      2775

Accuracy of model is 43.4954954954955 %


In [26]:
# Printing the evaluation scores
from sklearn.metrics import classification_report, confusion_matrix

# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix is\n")
print(cm)

print("\n")
print(classification_report(y_test, y_pred))

print("Accuracy of model is",accuracy_score(y_test,y_pred)*100,"%")

confusion matrix is

[[ 626 1075   18]
 [ 118  489   13]
 [  65  279   92]]


              precision    recall  f1-score   support

           0       0.77      0.36      0.50      1719
           1       0.27      0.79      0.40       620
           2       0.75      0.21      0.33       436

    accuracy                           0.43      2775
   macro avg       0.60      0.45      0.41      2775
weighted avg       0.66      0.43      0.45      2775

Accuracy of model is 43.4954954954955 %


# Decision Tree Classifier

In [28]:
# creating Decision Tree Classifier
dt=DecisionTreeClassifier(random_state=42,max_depth=3, min_samples_leaf=5)

In [29]:
# GridSearchcv for hyperparameter tuning

grid_param = {
    'criterion': ['gini', 'entropy']
}

In [30]:
gd_sr = GridSearchCV(estimator=dt,
                     param_grid=grid_param,
                     scoring='accuracy')

In [31]:
gd_sr.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(max_depth=3, min_samples_leaf=5,
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy']}, scoring='accuracy')

In [32]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'criterion': 'gini'}


In [33]:
best_result = gd_sr.best_score_
print(best_result*100,"%")

61.78804672180067 %


In [34]:
# Creating a model for best parameters

dt=DecisionTreeClassifier(criterion="gini", random_state=42,max_depth=3, min_samples_leaf=5)

In [35]:
dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=42)

In [36]:
y_pred=dt.predict(X_test)

In [37]:
import warnings
warnings.filterwarnings('ignore')

# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("confusion matrix is\n")
print(cm)

print("\n")
print(classification_report(y_test, y_pred))

print("Accuracy of model is",accuracy_score(y_test,y_pred)*100,"%")

confusion matrix is

[[1714    0    5]
 [ 617    0    3]
 [ 400    0   36]]


              precision    recall  f1-score   support

           0       0.63      1.00      0.77      1719
           1       0.00      0.00      0.00       620
           2       0.82      0.08      0.15       436

    accuracy                           0.63      2775
   macro avg       0.48      0.36      0.31      2775
weighted avg       0.52      0.63      0.50      2775

Accuracy of model is 63.06306306306306 %


# Implementing SVM

In [39]:
# Import svm model
from sklearn import svm
clf = svm.SVC(kernel='rbf') # RBF Kernel

In [40]:
clf.fit(X_train,y_train)

SVC()

In [41]:
y_pred=clf.predict(X_test)

In [42]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix is\n")
print(cm)

print("\n")
print(classification_report(y_test, y_pred))

print("Accuracy of model is",accuracy_score(y_test,y_pred)*100,"%")

confusion matrix is

[[1623   64   32]
 [ 478  114   28]
 [ 281   20  135]]


              precision    recall  f1-score   support

           0       0.68      0.94      0.79      1719
           1       0.58      0.18      0.28       620
           2       0.69      0.31      0.43       436

    accuracy                           0.67      2775
   macro avg       0.65      0.48      0.50      2775
weighted avg       0.66      0.67      0.62      2775

Accuracy of model is 67.45945945945945 %


In [43]:

rfc=RandomForestClassifier(n_estimators = 100, random_state = 0)

In [44]:
rfc.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [45]:
y_pred=rfc.predict(X_test)

In [46]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix is\n")
print(cm)

print("\n")
print(classification_report(y_test, y_pred))

print("Accuracy of model is",accuracy_score(y_test,y_pred)*100,"%")

confusion matrix is

[[1497  177   45]
 [ 350  228   42]
 [ 227   50  159]]


              precision    recall  f1-score   support

           0       0.72      0.87      0.79      1719
           1       0.50      0.37      0.42       620
           2       0.65      0.36      0.47       436

    accuracy                           0.68      2775
   macro avg       0.62      0.53      0.56      2775
weighted avg       0.66      0.68      0.66      2775

Accuracy of model is 67.89189189189189 %


In [48]:
# Predicting the label for an unknown text
test_data = "RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate"
print("Test data is\n", test_data, "\n")
test_data = preprocess_data(test_data)
print("Cleand data is\n", test_data, "\n")
Xt = vectorizer.transform([test_data])
Xt = Xt.toarray()
print("Vectorized test data is\n", Xt, "\n")
print("Size of the test data vector = ",np.shape(Xt), "\n")
yp = clf.predict(Xt)
#print(yp)
print("Predicted label = ", le.inverse_transform(yp)[0])

Test data is
 RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate 

Cleand data is
 everyone feel climate change question last night exactly 

Vectorized test data is
 [[0. 0. 0. ... 0. 0. 0.]] 

Size of the test data vector =  (1, 58376) 

Predicted label =  Neutral


### A1
* Logistic Regression & Random forest are performing good
### A2
* For Naive Bayes Classifier,
Accuracy obtained = 50%
For Logistic Regression,
Accuracy = 68.28%
* For kNN Algorithm,
No of Nearest Neighbors is HyperParameter. And n_neighbors=5, is giving better accuracy
Accuracy = 43.49%
* For Decision Tree Classifier,
Hyper parameter tuning is done
Accuracy = 63.06%
* For SVM,
Choosing kernel Hyperparameter.
Good accuracy is obtained for RBF Kernel
Accuracy = 67.45%
* For Random Forest Classifier,
Choosing No of Decison Tree Classifier is a hyperparameter
Accuracy = 67.8%
### A3
* In TF-IDF n_grams_range is an HyperParameter
used ngram_range=[1,1], [1,2]. Out of these, ngram_range=[1,2] giving better result
### A4
* Stemming is included in Preprocessing steps
with stemming, accuracy of models increased